# Calculating image metrics

In [1]:
import sys
sys.path.insert(0, '/home/oem/facediffusion')
from fdh256_dataset import FDF256Dataset

Could not load pyspng. Defaulting to pillow image backend.


/home/oem/anaconda3/envs/facediffusion/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset_path = '/home/oem/FDF/val'
dataset = FDF256Dataset(dirpath=dataset_path, load_keypoints=True, load_impath=True)

Dataset loaded from: /home/oem/FDF/val. Number of samples:6531


In [4]:
generated_ims_path = '/home/oem/facediffusion/results/ddim_generated_images_16'
sr_ims_path = '/home/oem/facediffusion/Coginfocom/ddim_generated_images_16-fdf'

In [7]:
from tqdm import tqdm
from skimage.metrics import structural_similarity as ssim
import tensorflow as tf
import torch
import os
import PIL
from PIL import Image
import numpy as np
import logging

logging.basicConfig(filename="DDIM_16",
                    filemode='a',
                    format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.INFO)
logging.info("Running image metrics comparison...")
logger = logging.getLogger('DDIM_16')

2023-11-21 21:56:15.930807: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-21 21:56:15.949932: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-21 21:56:15.949947: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-21 21:56:15.950491: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-21 21:56:15.953791: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-21 21:56:15.954297: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [9]:
total_pred_psnr = 0.0
total_pred_ssim = 0.0
total_sr_psnr = 0.0
total_sr_ssim = 0.0

for i in tqdm(range(len(dataset))):
    original_img = dataset[i]["img"]
    img_name: str = str(dataset[i]['impath']).split('/')[-1]
    img_idx = img_name.split('.')[0]
    
    pred_img_path = os.path.join(generated_ims_path, f'{img_idx}.jpg')
    pred_img = Image.open(pred_img_path)
    pred_img = pred_img.resize((256, 256), Image.BICUBIC)
    pred_img = np.asarray(pred_img)
    
    sr_img_path = os.path.join(sr_ims_path, f'{img_idx}.png')
    sr_img = Image.open(sr_img_path)
    sr_img = np.asarray(sr_img)
    
    pred_im_psnr: float = tf.image.psnr(pred_img, original_img, max_val=255)
    logger.info(f'{img_idx} PRED PSNR {pred_im_psnr}')
    pred_im_ssim: float = ssim(im1=original_img, im2=pred_img, data_range=pred_img.max() - pred_img.min(), channel_axis=2)
    logger.info(f'{img_idx} PRED SSIM {pred_im_ssim}')
    total_pred_psnr += pred_im_psnr
    total_pred_ssim += pred_im_ssim
    
    sr_im_psnr: float = tf.image.psnr(sr_img, original_img, max_val=255)
    logger.info(f'{img_idx} SR PSNR {sr_im_psnr}')
    sr_im_ssim: float = ssim(im1=original_img, im2=sr_img, data_range=sr_img.max() - sr_img.min(), channel_axis=2)
    logger.info(f'{img_idx} SR SSIM {sr_im_ssim}')
    total_sr_psnr += sr_im_psnr
    total_sr_ssim += sr_im_ssim
    
avg_pred_psnr = total_pred_psnr / len(dataset)
avg_pred_ssim = total_pred_ssim / len(dataset)

avg_sr_psnr = total_sr_psnr / len(dataset)
avg_sr_ssim = total_sr_ssim / len(dataset)

logger.info(f'Interpolated images PSNR = {avg_pred_psnr}')
logger.info(f'SR images PSNR = {avg_sr_psnr}')
logger.info(f'Interpolated images SSIM = {avg_pred_ssim}')
logger.info(f'SR images SSIM = {avg_sr_ssim}')

print(f'Interpolated images PSNR = {avg_pred_psnr}')
print(f'SR images PSNR = {avg_sr_psnr}')
print(f'Interpolated images SSIM = {avg_pred_ssim}')
print(f'SR images SSIM = {avg_sr_ssim}')


  0%|          | 0/6531 [00:00<?, ?it/s]2023-11-21 21:57:04.616042: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-21 21:57:04.619366: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
100%|██████████| 6531/6531 [02:28<00:00, 43.93it/s]

Interpolated images PSNR = 21.659893035888672
SR images PSNR = 19.727079391479492
Interpolated images SSIM = 0.675844280921999
SR images SSIM = 0.6085908290426706
